In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Mars
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат)
date_launch = Time("2028-12-24", scale="utc").tdb
date_flyby_Mars = Time("2029-04-30", scale="utc").tdb
date_flyby_Earth = Time("2031-02-11", scale="utc").tdb
date_arrival = Time("2034-08-18", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
mar = Ephem.from_body(Mars, time_range(date_launch, end=date_flyby_Mars))
earth_2 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Earth))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_mar = Orbit.from_ephem(Sun, mar, date_flyby_Mars)
ss_earth_2 = Orbit.from_ephem(Sun, earth_2, date_flyby_Earth)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_mar)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_mar, ss_earth_2)
ss_trans_2, ss_target_2 = ss_mar.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_earth_2, ss_jup)
ss_trans_3, ss_target_3 = ss_earth_2.apply_maneuver(man_lambert_3, intermediate=True)

In [6]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=101 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=65 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=193 * u.deg), color="black", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [7]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=101 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [8]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=65 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [9]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=193 * u.deg), color="crimson", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [10]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_mar, v_mar = Ephem.from_body(Mars, date_flyby_Mars).rv(date_flyby_Mars)
r_earth_2, v_earth_2 = Ephem.from_body(Earth, date_flyby_Earth).rv(date_flyby_Earth)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [11]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

-5716599.546472191 km
134699484.5463672 km
58393408.58919535 km

[-30.2490879   -1.19931253  -0.5199126 ] km / s


In [12]:
# Фазовые координаты Марса

print (f"{r_mar [0]}")
print (f"{r_mar [1]}")
print (f"{r_mar [2]}"'\n')

print (v_mar.to(u.km / u.s))

-226179074.4425368 km
-80275270.65798238 km
-30709046.837280877 km

[  9.48397594 -18.61643519  -8.79500365] km / s


In [13]:
# Фазовые координаты Земли_2

print (f"{r_earth_2 [0]}")
print (f"{r_earth_2 [1]}")
print (f"{r_earth_2 [2]}"'\n')

print (v_earth_2.to(u.km / u.s))

-115893867.80091257 km
84281275.8801818 km
36545675.72056785 km

[-18.99108526 -21.51442144  -9.32617477] km / s


In [14]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

738403429.2924684 km
57223250.847409785 km
6556876.238077645 km

[-1.13521071 12.52602907  5.39653309] km / s


In [15]:
nh_tof_1 = date_flyby_Mars - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_mar), = iod.lambert(Sun.k, r_earth, r_mar, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=127.000000020435>

In [16]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 6.40404852 km / s>

In [17]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [-6.21484835, -1.54504843, -0.01796638] km / s>

In [18]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end = (norm(vk_mar - v_mar)).to(u.km / u.s)
V_inf_end

<Quantity 13.22407506 km / s>

In [19]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end_vec = (vk_mar - v_mar).to(u.km / u.s)
V_inf_end_vec

<Quantity [-12.48475963,  -4.23382435,  -1.04003338] km / s>

In [20]:
nh_tof_2 = date_flyby_Earth - date_flyby_Mars # время полета Марс - Земля2

(v0_mar, vk_Ea2), = iod.lambert(Sun.k, r_mar, r_earth_2, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=651.999999994741>

In [21]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_vec_0 = (norm(v0_mar - v_mar)).to(u.km / u.s)
V_inf_end_vec_0

<Quantity 13.12002578 km / s>

In [22]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_vec_0_vec = (v0_mar - v_mar).to(u.km / u.s)
V_inf_end_vec_0_vec

<Quantity [-12.84512706,  -2.67139023,  -0.03823066] km / s>

In [41]:
# Разница гиперболических избытков скорости подлета и отлета 

v_gip_diff_1 = V_inf_end - V_inf_end_vec_0
v_gip_diff_1

<Quantity 0.10404928 km / s>

In [23]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2 = (norm(vk_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_end_2

<Quantity 11.59371694 km / s>

In [24]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2_vec = (vk_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_end_2_vec

<Quantity [-10.67611397,   3.87195277,   2.33299051] km / s>

In [25]:
nh_tof_3 = date_arrival - date_flyby_Earth # время полета Земля2 - Юпитер

(V0_Ea2, Vk_Jup), = iod.lambert(Sun.k, r_earth_2, r_jup, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=1283.9999999754732>

In [26]:
# Гиперболический избыток скорости при отлете от Земли_2

V_inf_end_2_0 = (norm(V0_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_end_2_0

<Quantity 11.59130429 km / s>

In [27]:
# Гиперболический избыток скорости при отлете от Земли_2

V_inf_end_2_0_vec = (V0_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_end_2_0_vec

<Quantity [  1.86662809, -10.93408959,  -3.36447909] km / s>

In [42]:
# Разница гиперболических избытков скорости подлета и отлета 

v_gip_diff_2 = V_inf_end_2 - V_inf_end_2_0
v_gip_diff_2

<Quantity 0.00241266 km / s>

In [28]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k = (norm(Vk_Jup - v_jup)).to(u.km / u.s)
V_inf_3_k

<Quantity 7.06500848 km / s>

In [29]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k_vec = (Vk_Jup - v_jup).to(u.km / u.s)
V_inf_3_k_vec

<Quantity [-3.07930447, -5.80519586, -2.59459626] km / s>

In [30]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 
time_adv

<TimeDelta object: scale='tdb' format='jd' value=2062.999999990649>

In [31]:
scalar_EM = r_earth[0]*r_mar[0] + r_earth[1]*r_mar[1] +r_earth[2]*r_mar[2] 
scalar_ME_2 = r_earth_2[0]*r_mar[0] + r_earth_2[1]*r_mar[1] +r_earth_2[2]*r_mar[2]
scalar_E2J = r_earth_2[0]*r_jup[0] + r_earth_2[1]*r_jup[1] +r_earth_2[2]*r_jup[2]

In [32]:
fi_EM = (scalar_EM/norm(r_earth)/norm(r_mar))
fi_ME2 = (scalar_ME_2/norm(r_earth_2)/norm(r_mar))
fi_E2J = (scalar_E2J/norm(r_earth_2)/norm(r_jup))

In [33]:
np.degrees(np.arccos(fi_EM))

<Quantity 108.55657812 deg>

In [34]:
np.degrees(np.arccos(fi_ME2))

<Quantity 59.19501735 deg>

In [35]:
np.degrees(np.arccos(fi_E2J))

<Quantity 137.31371529 deg>

In [36]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 6.4040485 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 4.83120157 km / s>

In [37]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 7.0650085

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

13.085695823986756
17.983782441612995


<Quantity -4.89808662 km / s>

In [38]:
## параметры участка Землия-Марс 

from math import*

to_rad = pi/180

ist_an_1 = 354.5456271 * to_rad
ist_an_2 = 102.3710450 * to_rad

exs_EM = (norm(r_mar)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_mar)/norm(r_earth) * cos(ist_an_2)))
p_EM = norm(r_earth)*(1+cos(ist_an_1)*exs_EM)
a_EM = p_EM/(1 - exs_EM*exs_EM)

print (f"Эксцентриситет {exs_EM}")
print (f"Фокальный параметр {p_EM}")
print (f"Большая полуось {a_EM}")

Эксцентриситет 0.479746934750368
Фокальный параметр 217089979.57294577 km
Большая полуось 281992580.05530274 km


In [39]:
## параметры участка Марс-Земля_2 

ist_an_1_ME2 = 120.3149260 * to_rad
ist_an_2_ME2 = 61.0044012 * to_rad

exs_ME2 = (norm(r_earth_2)/norm(r_mar) - 1) / (cos(ist_an_1_ME2) - (norm(r_earth_2)/norm(r_mar) * cos(ist_an_2_ME2)))
p_ME2 = norm(r_mar)*(1+cos(ist_an_1_ME2)*exs_ME2)
a_ME2 = p_ME2/(1 - exs_ME2*exs_ME2)

print (f"Эксцентриситет {exs_ME2}")
print (f"Фокальный параметр {p_ME2}")
print (f"Большая полуось {a_ME2}")

Эксцентриситет 0.4853716267609773
Фокальный параметр 182680816.076094 km
Большая полуось 238981395.3206977 km


In [72]:
## параметры участка Земля_2 - Юпитер

ist_an_1_E2J = 330.2065390 * to_rad 
ist_an_2_E2J = 192.7735255 * to_rad

exs_E2J = (norm(r_jup)/norm(r_earth_2) - 1) / (cos(ist_an_1_E2J) - (norm(r_jup)/norm(r_earth_2) * cos(ist_an_2_E2J)))
p_E2J = norm(r_earth_2)*(1+cos(ist_an_1_E2J)*exs_E2J)
a_E2J = p_E2J/(1 - exs_E2J*exs_E2J)

print (f"Эксцентриситет {exs_E2J}")
print (f"Фокальный параметр {p_E2J}")
print (f"Большая полуось {a_E2J}")

Эксцентриситет 0.6968276768632596
Фокальный параметр 237316206.37356904 km
Большая полуось 461317687.496406 km
